In [5]:
a360ai

<A360 AI Interface for project: Danish-test>

In [6]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [7]:
import warnings
warnings.filterwarnings('ignore')

### 1. Set Default Data Repo and load data

In [8]:
a360ai.set_default_datarepo("test")

In [9]:
a360ai.list_datasets()

,base_name,extension,size
0,Telco-Customer-Churn.csv,csv,977501


In [10]:
df = a360ai.load_dataset("Telco-Customer-Churn.csv")

In [11]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,...,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


### 2. Data cleaning, data engineering, data preprocessing

In [12]:
#Dropping the customerID as it has no use
df.drop('customerID',axis='columns',inplace=True)

In [13]:
df.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object

In [14]:
df.TotalCharges.values

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

In [15]:
# On viewing the datatypes, we see that TotalCharges is object, so we change it to integer
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()].shape

(11, 20)

In [16]:
#Remove rows with space in TotalCharges
df1 = df[df.TotalCharges!=" "]
df1.shape

(7032, 20)

In [17]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)
df1.TotalCharges.dtypes

dtype('float64')

In [18]:
# check the different values in the different columns
for column in df1:
  if df1[column].dtypes == object:  
    print(column+" :",df1[column].unique())

gender : ['Female' 'Male']
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
PhoneService : ['No' 'Yes']
MultipleLines : ['No phone service' 'No' 'Yes']
InternetService : ['DSL' 'Fiber optic' 'No']
OnlineSecurity : ['No' 'Yes' 'No internet service']
OnlineBackup : ['Yes' 'No' 'No internet service']
DeviceProtection : ['No' 'Yes' 'No internet service']
TechSupport : ['No' 'Yes' 'No internet service']
StreamingTV : ['No' 'Yes' 'No internet service']
StreamingMovies : ['No' 'Yes' 'No internet service']
Contract : ['Month-to-month' 'One year' 'Two year']
PaperlessBilling : ['Yes' 'No']
PaymentMethod : ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn : ['No' 'Yes']


In [19]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

In [20]:
# Converting all 'Yes' to 1 and all 'No' to 0
yes_no_columns = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity','OnlineBackup','DeviceProtection',
                  'TechSupport', 'StreamingTV', 'StreamingMovies','PaperlessBilling','Churn']
            
for col in yes_no_columns:
  df1[col].replace({'Yes':1,'No':0},inplace=True)

In [21]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

In [22]:
# One hot encoding for categorical columns
df2 = pd.get_dummies(data=df1,columns=['InternetService','Contract','PaymentMethod'])

In [23]:
cols = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()
df2[cols] = scaler.fit_transform(df2[cols])

In [24]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

### 3. Write dataaset to Data Repo

In [25]:
a360ai.write_dataset(X,"X")
a360ai.write_dataset(y,"y")

True